# Logistic regression model

## Recreating the logisitic regression from the paper using Sci-kit learn

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import string as str
%matplotlib inline

In [50]:
#import data
import pandas as pd

#Read in dataset with all rows included
df = pd.read_sas("/Users/roddy.jacques/Documents/DSA Project/Data/alldata3.sas7bdat")

#6931 DBD apps
dbd_apps = df[(df["eli_DBD"]==1)&(df["FAMILY_APPROACHED"]==2)]

#6060 DBD apps to match cohort in pape
dbd_apps = dbd_apps[(dbd_apps["eth_grp"]!=5)&(dbd_apps["FORMAL_APR_WHEN"]!=4)&(dbd_apps["donation_mentioned"]!=-1)
                    &(dbd_apps["FAMILY_WITNESS_BSDT"]!=9)&(dbd_apps["GENDER"]!=9)]
     
#9965 DCD apps
dcd_apps = df[(df["eli_DCD"]==1)&(df["FAMILY_APPROACHED"]==2)]

#9405 DCD apps to match cohort in paper
dcd_apps = dcd_apps[(dcd_apps["GENDER"]!=9)&(dcd_apps["cod_neuro"].notna())&(dcd_apps["eth_grp"]!=5)&(dcd_apps["donation_mentioned"]!=-1)&
                    (~dcd_apps["DTC_WD_TRTMENT_PRESENT"].isin([8,9]))]

/Users/roddy.jacques/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
/Users/roddy.jacques/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


In [66]:
# Columns used to create DBD model in paper
dbd_cols = ["wish", "FORMAL_APR_WHEN", "donation_mentioned", "app_nature", "eth_grp", "religion_grp", "GENDER", "FAMILY_WITNESS_BSDT", "DTC_PRESENT_BSD_CONV", 
            "acorn_new", "adult","FAMILY_CONSENT"]

dbd_apps[dbd_cols].astype(int)

dbd_model_data = dbd_apps[dbd_cols]
dbd_model_data2 = pd.get_dummies(data=dbd_model_data,columns=dbd_cols[:-1])

dbd_features = dbd_model_data2.drop("FAMILY_CONSENT",axis=1)
dbd_consents = dbd_model_data2["FAMILY_CONSENT"]

dbd_feature_names = dbd_features.columns.tolist()

In [80]:
LR_model = LogisticRegression(penalty='none')

LR_model.fit(dbd_features,dbd_consents)

odds_ratios = np.exp(LR_model.coef_) 

var_names = np.empty(39,dtype="U25")

for i in range(39):
    var_names[i] = dbd_feature_names[i].rsplit("_",1)[0]
var_names

array(['wish', 'wish', 'wish', 'wish', 'wish', 'FORMAL_APR_WHEN',
       'FORMAL_APR_WHEN', 'FORMAL_APR_WHEN', 'donation_mentioned',
       'donation_mentioned', 'donation_mentioned', 'donation_mentioned',
       'app_nature', 'app_nature', 'app_nature', 'eth_grp', 'eth_grp',
       'eth_grp', 'eth_grp', 'religion_grp', 'religion_grp',
       'religion_grp', 'religion_grp', 'religion_grp', 'religion_grp',
       'GENDER', 'GENDER', 'FAMILY_WITNESS_BSDT', 'FAMILY_WITNESS_BSDT',
       'DTC_PRESENT_BSD_CONV', 'DTC_PRESENT_BSD_CONV', 'acorn_new',
       'acorn_new', 'acorn_new', 'acorn_new', 'acorn_new', 'acorn_new',
       'adult', 'adult'], dtype='<U25')

In [84]:
cols_and_ors = np.column_stack([dbd_features.columns.tolist(),var_names,odds_ratios[0]])
cols_and_ors2 = pd.DataFrame(cols_and_ors)

cols_and_ors2.groupby(1

,0,1,2
0,wish_1.0,wish,0.19286004024379008
1,wish_2.0,wish,4.591863674646195
2,wish_3.0,wish,1.4613286364010425
3,wish_4.0,wish,3.563264061358002
4,wish_5.0,wish,0.29824758358901193
